In [1]:
#import dependencies

from _mempal_util import create_dirs, session_tasks
import nibabel as nib
import numpy as np
import os
import h5py
from sklearn import linear_model
from scipy import stats,special
import nibabel as nib
from nilearn.input_data import NiftiMasker,  MultiNiftiMasker


In [2]:
# REPLACE WITH YOUR LOCATION OF FMRIPREPed DATA
fmriprep_dir = '/jukebox/norman/rmasis/MemPal/data/BIDS/Norman/Baldassano/10442_vrpalace/derivatives/fmriprep'

### create subj_prefix list

subj_prefix = [('sub-sid{:0>2d}'.format(i)) for i in np.arange(1, 29+1)]
nSubj = len(subj_prefix)
print(nSubj, subj_prefix)

subj_sesh_dir = []
for subj in subj_prefix:
    subj_sesh_dir.append(subj + '/ses-01/func/')
    subj_sesh_dir.append(subj + '/ses-02/func/')
        
#create 
run_names = []
for subjIdx, subj in enumerate(subj_prefix):
    for task in session_tasks:
        run_names.append(subj + '_' + task)
        
fnames = []
confnames = []
for subjIdx, subj in enumerate(subj_prefix):
    for seshIdx, sesh in enumerate(session_tasks):
        for hem in ['L', 'R']: 
            confnames.append(os.path.join(subj, sesh.split('_')[0], 'func', subj + '_' + sesh + '_bold_confounds.tsv'))
            fnames.append(os.path.join(subj, sesh.split('_')[0], 'func', subj + '_' + sesh + '_bold_space-fsaverage6.'+ hem +'.func.gii'))
               
                
fnames.sort() #put things alphabetically    
run_names = np.repeat(run_names,2) #make run_names same length as fnames and confnames

29 ['sub-sid01', 'sub-sid02', 'sub-sid03', 'sub-sid04', 'sub-sid05', 'sub-sid06', 'sub-sid07', 'sub-sid08', 'sub-sid09', 'sub-sid10', 'sub-sid11', 'sub-sid12', 'sub-sid13', 'sub-sid14', 'sub-sid15', 'sub-sid16', 'sub-sid17', 'sub-sid18', 'sub-sid19', 'sub-sid20', 'sub-sid21', 'sub-sid22', 'sub-sid23', 'sub-sid24', 'sub-sid25', 'sub-sid26', 'sub-sid27', 'sub-sid28', 'sub-sid29']


In [3]:
run_names

array(['sub-sid01_ses-01_task-pathvideo',
       'sub-sid01_ses-01_task-pathvideo',
       'sub-sid01_ses-01_task-roomvideo_run-01',
       'sub-sid01_ses-01_task-roomvideo_run-01',
       'sub-sid01_ses-01_task-roomvideo_run-02',
       'sub-sid01_ses-01_task-roomvideo_run-02',
       'sub-sid01_ses-02_task-pathvideo',
       'sub-sid01_ses-02_task-pathvideo', 'sub-sid01_ses-02_task-recall',
       'sub-sid01_ses-02_task-recall',
       'sub-sid01_ses-02_task-roomobjectvideo_run-01',
       'sub-sid01_ses-02_task-roomobjectvideo_run-01',
       'sub-sid01_ses-02_task-roomobjectvideo_run-02',
       'sub-sid01_ses-02_task-roomobjectvideo_run-02',
       'sub-sid02_ses-01_task-pathvideo',
       'sub-sid02_ses-01_task-pathvideo',
       'sub-sid02_ses-01_task-roomvideo_run-01',
       'sub-sid02_ses-01_task-roomvideo_run-01',
       'sub-sid02_ses-01_task-roomvideo_run-02',
       'sub-sid02_ses-01_task-roomvideo_run-02',
       'sub-sid02_ses-02_task-pathvideo',
       'sub-sid02_ses-0

In [14]:
#PREPROCESSING (MOTION CORRECTION AND .....)

processed_dir = '../PythonData2024/ProcessedData/' ; create_dirs(processed_dir)

regs = [] #accumulated regs
regrs = []
errors = []

D = dict()

for file,conf,run in zip(fnames, confnames, run_names):
    
    try: 
    
        print('***{}***'.format(run.split('_')[0]))

        print('    Processing run ', file.split('/')[-1])

        if 'L.func.gii' in file:
            hem = 'L'
        elif 'R.func.gii' in file:
            hem = 'R'

        print('    Loading ', os.path.join(fmriprep_dir , file))
        gi = nib.load(os.path.join(fmriprep_dir , file))

        D[hem] = np.column_stack([gi.darrays[t].data for t in range(len(gi.darrays))])

        # Use regressors for:
        # -CSF
        # -WhiteMatter
        # -FramewiseDisplacement
        # -All cosine bases for drift (0.008 Hz = 125s)
        # -X, Y, Z and derivatives
        # -RotX, RotY, RotZ and derivatives

        #confounds
        print('    Extracting confound information from ', conf)
        conf = np.genfromtxt(os.path.join(fmriprep_dir ,conf), names=True)

        #motion correction needs to occur on all 3d axeses
        motion = np.column_stack((conf['X'],
          conf['Y'],
          conf['Z'],
          conf['RotX'],
          conf['RotY'],
          conf['RotZ']))


        #regressors list
        reg = np.column_stack((conf['CSF'],
               conf['WhiteMatter'],
               np.nan_to_num(conf['FramewiseDisplacement']),
               np.column_stack([conf[k] for k in conf.dtype.names if 'Cosine' in k]),
               motion,
               np.vstack((np.zeros((1,motion.shape[1])), np.diff(motion, axis=0)))))

        regs.append(reg)

        #create linear regression object
        regr = linear_model.LinearRegression()

        regrs.append(regr)

        #train with regressors
        #reg : 595x26 ; D[hem] : 40962x595 ; D[hem].T : 595x40962 where 595 = n_samples, 26=n_features/explanatory, 40962=n_targets
        regr.fit(reg, D[hem].T) 

        #use coeffs generated from fit
        print('   D[hem] = ', np.shape(D[hem]))
        print('   np dot regrcoef, reg.t: ', np.shape(np.dot(regr.coef_, reg.T)) )
        print('   interecept shape: ', np.shape(regr.intercept_[:, np.newaxis]))

        #subtract the nuissance regressors from the data (https://www.sciencedirect.com/science/article/pii/S1053811916307418)
        D[hem] = D[hem] - np.dot(regr.coef_, reg.T) - regr.intercept_[:, np.newaxis]

        #z-score (center the data) across the timeseries dimension;
        D[hem] = stats.zscore(D[hem], axis=1)

        subj = file.split('/')[0]
        sesh = file.split('/')[1]

        #check if we have a subj-specific directory
        if os.path.exists(processed_dir + subj):
            #print('   ../PythonData/{}/ already exists.'.format(subj))
            subj
        else:
            os.mkdir(processed_dir + subj) #if no directory for this subject, make one!
            print('   making new directory {}/{}/'.format(processed_dir,subj))

        if hem == 'L': #make sure to manually create subject folders in PythonData or add snippet to check for existence/create
            print("...saving left hem.")
            with h5py.File(os.path.join(processed_dir,subj, run + '.h5'), 'w') as hf:
                hf.create_dataset('L', data=D['L'])
        elif hem == 'R':
            print("...saving right hem.")
            with h5py.File(os.path.join(processed_dir,subj, run + '.h5'), 'a') as hf:
                hf.create_dataset('R', data=D['R'])
                
    except Exception as e: 
        print('**** ERROR ****')
        print('**** ERROR ****')
        print('---something happened---')
        print(e)
        errors.append(e)
        print('**** ERROR ****')
        print('**** ERROR ****')
        



print('Finished cleaning and prepping left and right hemispheres per run.')       

Folder already exists: ../PythonData2024/ProcessedData/
***sub-sid01***
    Processing run  sub-sid01_ses-01_task-pathvideo_bold_space-fsaverage6.L.func.gii
    Loading  /jukebox/norman/rmasis/MemPal/data/BIDS/Norman/Baldassano/10442_vrpalace/derivatives/fmriprep/sub-sid01/ses-01/func/sub-sid01_ses-01_task-pathvideo_bold_space-fsaverage6.L.func.gii
    Extracting confound information from  sub-sid01/ses-01/func/sub-sid01_ses-01_task-pathvideo_bold_confounds.tsv
   D[hem] =  (40962, 595)
   np dot regrcoef, reg.t:  (40962, 595)
   interecept shape:  (40962, 1)


/jukebox/pkgs/PYGER/0.9.1/lib/python3.7/site-packages/scipy/stats/stats.py:2279: RuntimeWarning: divide by zero encountered in true_divide
  np.expand_dims(sstd, axis=axis))
/jukebox/pkgs/PYGER/0.9.1/lib/python3.7/site-packages/scipy/stats/stats.py:2279: RuntimeWarning: invalid value encountered in true_divide
  np.expand_dims(sstd, axis=axis))


   making new directory ../PythonData2024/ProcessedData//sub-sid01/
...saving left hem.
***sub-sid01***
    Processing run  sub-sid01_ses-01_task-pathvideo_bold_space-fsaverage6.R.func.gii
    Loading  /jukebox/norman/rmasis/MemPal/data/BIDS/Norman/Baldassano/10442_vrpalace/derivatives/fmriprep/sub-sid01/ses-01/func/sub-sid01_ses-01_task-pathvideo_bold_space-fsaverage6.R.func.gii
    Extracting confound information from  sub-sid01/ses-01/func/sub-sid01_ses-01_task-pathvideo_bold_confounds.tsv
   D[hem] =  (40962, 595)
   np dot regrcoef, reg.t:  (40962, 595)
   interecept shape:  (40962, 1)


/jukebox/pkgs/PYGER/0.9.1/lib/python3.7/site-packages/scipy/stats/stats.py:2279: RuntimeWarning: divide by zero encountered in true_divide
  np.expand_dims(sstd, axis=axis))
/jukebox/pkgs/PYGER/0.9.1/lib/python3.7/site-packages/scipy/stats/stats.py:2279: RuntimeWarning: invalid value encountered in true_divide
  np.expand_dims(sstd, axis=axis))


...saving right hem.
***sub-sid01***
    Processing run  sub-sid01_ses-01_task-roomvideo_run-01_bold_space-fsaverage6.L.func.gii
    Loading  /jukebox/norman/rmasis/MemPal/data/BIDS/Norman/Baldassano/10442_vrpalace/derivatives/fmriprep/sub-sid01/ses-01/func/sub-sid01_ses-01_task-roomvideo_run-01_bold_space-fsaverage6.L.func.gii
    Extracting confound information from  sub-sid01/ses-01/func/sub-sid01_ses-01_task-roomvideo_run-01_bold_confounds.tsv
   D[hem] =  (40962, 235)
   np dot regrcoef, reg.t:  (40962, 235)
   interecept shape:  (40962, 1)


...

...saving right hem.
***sub-sid29***
    Processing run  sub-sid29_ses-02_task-roomobjectvideo_run-02_bold_space-fsaverage6.L.func.gii
    Loading  /jukebox/norman/rmasis/MemPal/data/BIDS/Norman/Baldassano/10442_vrpalace/derivatives/fmriprep/sub-sid29/ses-02/func/sub-sid29_ses-02_task-roomobjectvideo_run-02_bold_space-fsaverage6.L.func.gii
    Extracting confound information from  sub-sid29/ses-02/func/sub-sid29_ses-02_task-roomobjectvideo_run-02_bold_confounds.tsv
   D[hem] =  (40962, 460)
   np dot regrcoef, reg.t:  (40962, 460)
   interecept shape:  (40962, 1)


/jukebox/pkgs/PYGER/0.9.1/lib/python3.7/site-packages/scipy/stats/stats.py:2279: RuntimeWarning: divide by zero encountered in true_divide
  np.expand_dims(sstd, axis=axis))
/jukebox/pkgs/PYGER/0.9.1/lib/python3.7/site-packages/scipy/stats/stats.py:2279: RuntimeWarning: invalid value encountered in true_divide
  np.expand_dims(sstd, axis=axis))


...saving left hem.
***sub-sid29***
    Processing run  sub-sid29_ses-02_task-roomobjectvideo_run-02_bold_space-fsaverage6.R.func.gii
    Loading  /jukebox/norman/rmasis/MemPal/data/BIDS/Norman/Baldassano/10442_vrpalace/derivatives/fmriprep/sub-sid29/ses-02/func/sub-sid29_ses-02_task-roomobjectvideo_run-02_bold_space-fsaverage6.R.func.gii
    Extracting confound information from  sub-sid29/ses-02/func/sub-sid29_ses-02_task-roomobjectvideo_run-02_bold_confounds.tsv
   D[hem] =  (40962, 460)
   np dot regrcoef, reg.t:  (40962, 460)
   interecept shape:  (40962, 1)


/jukebox/pkgs/PYGER/0.9.1/lib/python3.7/site-packages/scipy/stats/stats.py:2279: RuntimeWarning: divide by zero encountered in true_divide
  np.expand_dims(sstd, axis=axis))
/jukebox/pkgs/PYGER/0.9.1/lib/python3.7/site-packages/scipy/stats/stats.py:2279: RuntimeWarning: invalid value encountered in true_divide
  np.expand_dims(sstd, axis=axis))


...saving right hem.
Finished cleaning and prepping left and right hemispheres per run.
